####Insttaller set up

In [ ]:
!pip install replicate gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2




### Base  codes


In [ ]:
import replicate
import gradio as gr
import os
import requests
from PIL import Image
from io import BytesIO
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set your Replicate API token
os.environ["REPLICATE_API_TOKEN"] = "my_token"

def generate_image(prompt):
    try:
        logger.info(f"Starting image generation with prompt: {prompt}")

        if not prompt:
            raise ValueError("Prompt cannot be empty")

        input = {
            "prompt": prompt
        }

        logger.info("Making API call to Replicate...")
        output = replicate.run(
            "ideogram-ai/ideogram-v2-turbo",
            input=input
        )

        logger.info(f"API Response received: {output}")

        # Convert FileOutput to bytes
        if hasattr(output, 'read'):
            image_data = output.read()
            return image_data

        return None

    except Exception as e:
        logger.error(f"Error in generate_image: {str(e)}")
        return None

def generate_and_display(prompt):
    try:
        logger.info("Starting generate_and_display function")
        image_data = generate_image(prompt)

        if not image_data:
            logger.error("No image data received")
            return None, "Failed to generate image. Please try again."

        # Convert bytes directly to image
        img = Image.open(BytesIO(image_data))
        logger.info("Successfully processed image")
        return img, "Image generated successfully!"

    except Exception as e:
        error_msg = str(e)
        logger.error(f"Error in generate_and_display: {error_msg}")
        return None, f"Error: {error_msg}"

# Gradio interface setup
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🎨 AI Image Generator")

    with gr.Row():
        prompt_input = gr.Textbox(
            label="Enter your detailed prompt",
            placeholder="Describe your image with specific details",
            lines=4
        )

    # Example prompts
    example_prompts = [
        "An illustration of a gold running shoe with the text \"Run AI with an API\" written on the shoe. The shoe is placed on a pink background. The text is white and bold. The overall image has a modern and techy vibe, with elements of speed.",
        "A vintage green delivery van with 'KLASEN' text on the side, photographed in a modern city setting with dramatic lighting",
        "A sleek green glass bottle with a black cap, placed on a minimalist white surface with soft shadows and studio lighting"
    ]

    gr.Examples(
        examples=example_prompts,
        inputs=prompt_input,
        label="Example Prompts - Click to try!"
    )

    with gr.Row():
        generate_button = gr.Button("Generate Image", variant="primary")
        image_output = gr.Image(label="Generated Image")
        status_output = gr.Textbox(label="Status", interactive=False)

    generate_button.click(
        fn=generate_and_display,
        inputs=[prompt_input],
        outputs=[image_output, status_output]
    )

if __name__ == "__main__":
    demo.launch(show_error=True)

###Parameterized code version

#####edited coorected  version  debugger added hand written

#####New  code ideogram-V2- turbo

In [ ]:
import replicate
import gradio as gr
import os
import requests
from PIL import Image
from io import BytesIO
import logging
import json

# Enhanced logging setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Set your Replicate API token
os.environ["REPLICATE_API_TOKEN"] = "my_token"

def construct_prompt(base_prompt, style, mood, lighting, camera_angle, background, color_scheme):
    """
    Constructs a detailed prompt from the base prompt and additional parameters
    """
    try:
        if not base_prompt:
            raise ValueError("Base prompt is required")

        enhanced_prompt = f"{base_prompt}"
        if style:
            enhanced_prompt += f", {style} style"
        if lighting:
            enhanced_prompt += f", with {lighting} lighting"
        if camera_angle:
            enhanced_prompt += f", {camera_angle} perspective"
        if background:
            enhanced_prompt += f", {background} background"
        if color_scheme:
            enhanced_prompt += f", emphasizing {color_scheme} tones"
        if mood:
            enhanced_prompt += f", creating a {mood} mood"

        logger.info(f"Constructed prompt: {enhanced_prompt}")
        return enhanced_prompt
    except Exception as e:
        logger.error(f"Error constructing prompt: {str(e)}")
        raise

def prepare_api_parameters(prompt, resolution=None):
    """
    Prepares and validates API parameters
    """
    try:
        params = {
            "prompt": prompt,
        }

        if resolution and resolution.lower() != "none":
            try:
                width, height = map(int, resolution.split('x'))
                params["width"] = width
                params["height"] = height
                logger.info(f"Added resolution parameters: {width}x{height}")
            except ValueError:
                logger.warning(f"Invalid resolution format: {resolution}. Skipping resolution.")

        logger.info(f"Prepared API parameters: {json.dumps(params, indent=2)}")
        return params
    except Exception as e:
        logger.error(f"Error preparing API parameters: {str(e)}")
        raise

def process_api_response(output):
    """
    Processes and validates API response
    """
    try:
        logger.info(f"Processing API response: {output}")

        if not output:
            logger.error("Empty response from API")
            return None

        if isinstance(output, list) and len(output) > 0:
            image_url = output[0]
            logger.info(f"Extracted image URL: {image_url}")
            return image_url
        elif hasattr(output, 'read'):
            logger.info("Received file-like object from API")
            return output
        else:
            logger.error(f"Unexpected response type: {type(output)}")
            return None
    except Exception as e:
        logger.error(f"Error processing API response: {str(e)}")
        return None

def fetch_image(image_url):
    """
    Fetches the image from the provided URL
    """
    try:
        logger.info(f"Fetching image from URL: {image_url}")
        response = requests.get(image_url, timeout=10)
        response.raise_for_status()
        logger.info("Successfully fetched image")
        return response.content
    except requests.RequestException as e:
        logger.error(f"Error fetching image: {str(e)}")
        return None

def generate_image(base_prompt, resolution, aspect_ratio, color_scheme, style,
                   mood, lighting, camera_angle, background):
    """
    Main image generation function
    """
    try:
        logger.info("Starting image generation...")

        # Construct the enhanced prompt
        enhanced_prompt = construct_prompt(
            base_prompt, style, mood, lighting,
            camera_angle, background, color_scheme
        )

        # Prepare API parameters
        input_params = prepare_api_parameters(enhanced_prompt, resolution)

        # Make API call
        logger.info("Making API call to Replicate...")
        output = replicate.run(
            "ideogram-ai/ideogram-v2-turbo",
            input=input_params
        )

        # Process API response
        image_url = process_api_response(output)
        if not image_url:
            logger.error("No valid image URL returned from API")
            return None

        # Fetch image
        image_data = fetch_image(image_url)
        if not image_data:
            logger.error("Failed to fetch image data")
            return None

        return image_data
    except Exception as e:
        logger.error(f"Error in generate_image: {str(e)}")
        return None

def generate_and_display(base_prompt, resolution, aspect_ratio, color_scheme, style,
                         mood, lighting, camera_angle, background):
    """
    Wrapper function for Gradio to generate and display the image
    """
    try:
        logger.info("Starting generate_and_display function")

        if not base_prompt:
            return None, "Error: Base prompt is required"

        # Generate image
        image_data = generate_image(
            base_prompt, resolution, aspect_ratio, color_scheme,
            style, mood, lighting, camera_angle, background
        )

        if not image_data:
            logger.error("No image data received from generation")
            return None, "Failed to generate image. Please check logs for more details."

        # Convert to PIL Image
        img = Image.open(BytesIO(image_data))
        logger.info("Successfully converted image data to PIL Image")
        return img, "Image generated successfully!"
    except Exception as e:
        error_msg = str(e)
        logger.error(f"Error converting image data: {error_msg}")
        return None, f"Error processing image: {error_msg}"

# Gradio interface setup
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🎨 Parameterized AI Image Generator with Base Prompt")

    with gr.Row():
        base_prompt_input = gr.Textbox(
            label="Base Prompt",
            placeholder="Describe your base image concept",
            lines=2
        )
        style_input = gr.Dropdown(
            label="Art Style",
            choices=["Realistic", "Digital Art", "3D Render", "Pencil Sketch"],
            value="Realistic"
        )
        resolution_input = gr.Dropdown(
            label="Resolution",
            choices=["None", "512x512", "1024x1024", "2048x2048"],
            value="1024x1024"
        )
        aspect_ratio_input = gr.Dropdown(
            label="Aspect Ratio",
            choices=["1:1", "16:9", "9:16", "3:2"],
            value="1:1"
        )
        color_input = gr.Textbox(
            label="Color Scheme",
            placeholder="e.g., metallic, red, blue"
        )
        mood_input = gr.Textbox(
            label="Mood",
            placeholder="e.g., dramatic, peaceful, modern"
        )
        lighting_input = gr.Textbox(
            label="Lighting",
            placeholder="e.g., natural, studio, dramatic"
        )
        camera_angle_input = gr.Textbox(
            label="Camera Angle",
            placeholder="e.g., front view, side view, aerial view"
        )
        background_input = gr.Textbox(
            label="Background",
            placeholder="e.g., forest, galaxy, abstract"
        )

    generate_button = gr.Button("Generate Image")
    image_output = gr.Image(label="Generated Image")
    status_output = gr.Textbox(label="Status", interactive=False)

    generate_button.click(
        fn=generate_and_display,
        inputs=[
            base_prompt_input,
            resolution_input,
            aspect_ratio_input,
            color_input,
            style_input,
            mood_input,
            lighting_input,
            camera_angle_input,
            background_input,
        ],
        outputs=[image_output, status_output]
    )

if __name__ == "__main__":
    demo.launch(show_error=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e497500a654f1af26c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#####upgraded coe with  validation statements applied to base prompt

In [ ]:
import replicate
import gradio as gr
import os
import requests
from PIL import Image
from io import BytesIO
import logging
import json

# Enhanced logging setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Set your Replicate API token
os.environ["REPLICATE_API_TOKEN"] = "my_token"

def construct_prompt(base_prompt, style, mood, lighting, camera_angle, background, color_scheme):
    """
    Constructs a detailed prompt from the base prompt and additional parameters
    """
    try:
        if not base_prompt:
            raise ValueError("Base prompt is required")

        enhanced_prompt = f"{base_prompt}"
        if style:
            enhanced_prompt += f", {style} style"
        if lighting:
            enhanced_prompt += f", with {lighting} lighting"
        if camera_angle:
            enhanced_prompt += f", {camera_angle} perspective"
        if background:
            enhanced_prompt += f", {background} background"
        if color_scheme:
            enhanced_prompt += f", emphasizing {color_scheme} tones"
        if mood:
            enhanced_prompt += f", creating a {mood} mood"

        logger.info(f"Constructed prompt: {enhanced_prompt}")
        return enhanced_prompt
    except Exception as e:
        logger.error(f"Error constructing prompt: {str(e)}")
        raise

def prepare_api_parameters(prompt, resolution=None):
    """
    Prepares and validates API parameters
    """
    try:
        params = {
            "prompt": prompt,
        }

        if resolution and resolution.lower() != "none":
            try:
                width, height = map(int, resolution.split('x'))
                params["width"] = width
                params["height"] = height
                logger.info(f"Added resolution parameters: {width}x{height}")
            except ValueError:
                logger.warning(f"Invalid resolution format: {resolution}. Skipping resolution.")

        logger.info(f"Prepared API parameters: {json.dumps(params, indent=2)}")
        return params
    except Exception as e:
        logger.error(f"Error preparing API parameters: {str(e)}")
        raise

def process_api_response(output):
    """
    Processes and validates API response
    """
    try:
        logger.info(f"Processing API response: {output}")

        if not output:
            logger.error("Empty response from API")
            return None

        if isinstance(output, list) and len(output) > 0:
            image_url = output[0]
            logger.info(f"Extracted image URL: {image_url}")
            return image_url
        elif hasattr(output, 'read'):
            logger.info("Received file-like object from API")
            return output
        else:
            logger.error(f"Unexpected response type: {type(output)}")
            return None
    except Exception as e:
        logger.error(f"Error processing API response: {str(e)}")
        return None

def fetch_image(image_url):
    """
    Fetches the image from the provided URL
    """
    try:
        logger.info(f"Fetching image from URL: {image_url}")
        response = requests.get(image_url, timeout=10)
        response.raise_for_status()
        logger.info("Successfully fetched image")
        return response.content
    except requests.RequestException as e:
        logger.error(f"Error fetching image: {str(e)}")
        return None

def generate_image(base_prompt, resolution, aspect_ratio, color_scheme, style,
                   mood, lighting, camera_angle, background):
    """
    Main image generation function
    """
    try:
        logger.info("Starting image generation...")

        # Construct the enhanced prompt
        enhanced_prompt = construct_prompt(
            base_prompt, style, mood, lighting,
            camera_angle, background, color_scheme
        )

        # Prepare API parameters
        input_params = prepare_api_parameters(enhanced_prompt, resolution)

        # Make API call
        logger.info("Making API call to Replicate...")
        output = replicate.run(
            "ideogram-ai/ideogram-v2-turbo",
            input=input_params
        )

        # Process API response
        image_url = process_api_response(output)
        if not image_url:
            logger.error("No valid image URL returned from API")
            return None

        # Fetch image
        image_data = fetch_image(image_url)
        if not image_data:
            logger.error("Failed to fetch image data")
            return None

        return image_data
    except Exception as e:
        logger.error(f"Error in generate_image: {str(e)}")
        return None

# Enhanced logging setup remains the same...

def validate_base_prompt(prompt):
    """
    Validates the base prompt and provides suggestions for improvement
    """
    if not prompt or prompt.strip() == "":
        return False, "Base prompt cannot be empty. Please describe what you want to generate."

    # Check minimum word count (recommended at least 3 words)
    words = prompt.split()
    if len(words) < 3:
        return False, "Base prompt is too short. Try adding more details (at least 3 words recommended)."

    # Check for common issues
    common_issues = []

    # Check for specific object/subject
    if not any(word.lower() in prompt.lower() for word in ['spaceship', 'car', 'person', 'building', 'landscape', 'animal']):
        common_issues.append("Consider specifying the main subject (e.g., spaceship, person, landscape)")

    # Check for descriptive adjectives
    descriptive_words = ['red', 'large', 'modern', 'ancient', 'futuristic', 'beautiful', 'dark', 'bright']
    if not any(word.lower() in prompt.lower() for word in descriptive_words):
        common_issues.append("Add descriptive adjectives (e.g., modern, futuristic, ancient)")

    # If there are issues but prompt is not invalid
    if common_issues and len(words) >= 3:
        return True, "Suggestions for better results:\n- " + "\n- ".join(common_issues)

    return True, "Prompt looks good!"


def generate_and_display(base_prompt, resolution, aspect_ratio, color_scheme, style,
                         mood, lighting, camera_angle, background):
    """
    Wrapper function for Gradio to generate and display the image
    """
    try:
        logger.info("Starting generate_and_display function")

        if not base_prompt:
            return None, "Error: Base prompt is required"

        # Generate image
        image_data = generate_image(
            base_prompt, resolution, aspect_ratio, color_scheme,
            style, mood, lighting, camera_angle, background
        )

        if not image_data:
            logger.error("No image data received from generation")
            return None, "Failed to generate image. Please check logs for more details."

        # Convert to PIL Image
        img = Image.open(BytesIO(image_data))
        logger.info("Successfully converted image data to PIL Image")
        return img, "Image generated successfully!"
    except Exception as e:
        error_msg = str(e)
        logger.error(f"Error converting image data: {error_msg}")
        return None, f"Error processing image: {error_msg}"


# Gradio interface setup
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🎨 Parameterized AI Image Generator with Base Prompt")

    with gr.Row():
        base_prompt_input = gr.Textbox(
            label="Base Prompt",
            placeholder="Describe your base image concept",
            lines=2
        )

    # Define prompt_feedback before using it
    prompt_feedback = gr.Textbox(
        label="Prompt Feedback",
        interactive=False
    )

    with gr.Row():
        style_input = gr.Dropdown(
            label="Art Style",
            choices=["Realistic", "Digital Art", "3D Render", "Pencil Sketch"],
            value="Realistic"
        )
        resolution_input = gr.Dropdown(
            label="Resolution",
            choices=["None", "512x512", "1024x1024", "2048x2048"],
            value="1024x1024"
        )
        aspect_ratio_input = gr.Dropdown(
            label="Aspect Ratio",
            choices=["1:1", "16:9", "9:16", "3:2"],
            value="1:1"
        )

    with gr.Row():
        color_input = gr.Textbox(
            label="Color Scheme",
            placeholder="e.g., metallic, red, blue"
        )
        mood_input = gr.Textbox(
            label="Mood",
            placeholder="e.g., dramatic, peaceful, modern"
        )
        lighting_input = gr.Textbox(
            label="Lighting",
            placeholder="e.g., natural, studio, dramatic"
        )

    with gr.Row():
        camera_angle_input = gr.Textbox(
            label="Camera Angle",
            placeholder="e.g., front view, side view, aerial view"
        )
        background_input = gr.Textbox(
            label="Background",
            placeholder="e.g., forest, galaxy, abstract"
        )

    def update_prompt_feedback(prompt):
        is_valid, message = validate_base_prompt(prompt)
        return message

    base_prompt_input.change(
        fn=update_prompt_feedback,
        inputs=[base_prompt_input],
        outputs=[prompt_feedback]
    )

    generate_button = gr.Button("Generate Image")
    image_output = gr.Image(label="Generated Image")
    status_output = gr.Textbox(label="Status", interactive=False)

    generate_button.click(
        fn=generate_and_display,
        inputs=[
            base_prompt_input,
            resolution_input,
            aspect_ratio_input,
            color_input,
            style_input,
            mood_input,
            lighting_input,
            camera_angle_input,
            background_input,
        ],
        outputs=[image_output, status_output]
    )

if __name__ == "__main__":
    demo.launch(show_error=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cb247c5efeaeb246f9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


With  caption on the  text  ovelay on  image  with  positional specification

In [ ]:
import replicate
import gradio as gr
import os
import requests
from PIL import Image
from io import BytesIO
import logging
import json

# Enhanced logging setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Set your Replicate API token
os.environ["REPLICATE_API_TOKEN"] = "my_token"

def construct_prompt(base_prompt, style, mood, lighting, camera_angle, background, color_scheme):
    """
    Constructs a detailed prompt from the base prompt and additional parameters
    """
    try:
        if not base_prompt:
            raise ValueError("Base prompt is required")

        enhanced_prompt = f"{base_prompt}"
        if style:
            enhanced_prompt += f", {style} style"
        if lighting:
            enhanced_prompt += f", with {lighting} lighting"
        if camera_angle:
            enhanced_prompt += f", {camera_angle} perspective"
        if background:
            enhanced_prompt += f", {background} background"
        if color_scheme:
            enhanced_prompt += f", emphasizing {color_scheme} tones"
        if mood:
            enhanced_prompt += f", creating a {mood} mood"

        logger.info(f"Constructed prompt: {enhanced_prompt}")
        return enhanced_prompt
    except Exception as e:
        logger.error(f"Error constructing prompt: {str(e)}")
        raise

def prepare_api_parameters(prompt, resolution=None):
    """
    Prepares and validates API parameters
    """
    try:
        params = {
            "prompt": prompt,
        }

        if resolution and resolution.lower() != "none":
            try:
                width, height = map(int, resolution.split('x'))
                params["width"] = width
                params["height"] = height
                logger.info(f"Added resolution parameters: {width}x{height}")
            except ValueError:
                logger.warning(f"Invalid resolution format: {resolution}. Skipping resolution.")

        logger.info(f"Prepared API parameters: {json.dumps(params, indent=2)}")
        return params
    except Exception as e:
        logger.error(f"Error preparing API parameters: {str(e)}")
        raise

def process_api_response(output):
    """
    Processes and validates API response
    """
    try:
        logger.info(f"Processing API response: {output}")

        if not output:
            logger.error("Empty response from API")
            return None

        if isinstance(output, list) and len(output) > 0:
            image_url = output[0]
            logger.info(f"Extracted image URL: {image_url}")
            return image_url
        elif hasattr(output, 'read'):
            logger.info("Received file-like object from API")
            return output
        else:
            logger.error(f"Unexpected response type: {type(output)}")
            return None
    except Exception as e:
        logger.error(f"Error processing API response: {str(e)}")
        return None

def fetch_image(image_url):
    """
    Fetches the image from the provided URL
    """
    try:
        logger.info(f"Fetching image from URL: {image_url}")
        response = requests.get(image_url, timeout=10)
        response.raise_for_status()
        logger.info("Successfully fetched image")
        return response.content
    except requests.RequestException as e:
        logger.error(f"Error fetching image: {str(e)}")
        return None

def generate_image(base_prompt, resolution, aspect_ratio, color_scheme, style,
                   mood, lighting, camera_angle, background, text_overlay=None, text_position="center"):
    """
    Main image generation function with text overlay support
    """
    try:
        logger.info("Starting image generation...")

        # Construct the enhanced prompt
        enhanced_prompt = construct_prompt(
            base_prompt, style, mood, lighting,
            camera_angle, background, color_scheme
        )

        # Add text overlay to prompt if provided
        if text_overlay:
            enhanced_prompt += f", with text '{text_overlay}' overlaid on the {text_position}"

        # Prepare API parameters
        input_params = prepare_api_parameters(enhanced_prompt, resolution)

        # Make API call
        logger.info("Making API call to Replicate...")
        output = replicate.run(
            "ideogram-ai/ideogram-v2-turbo",
            input=input_params
        )

        # Process API response
        image_url = process_api_response(output)
        if not image_url:
            logger.error("No valid image URL returned from API")
            return None

        # Fetch image
        image_data = fetch_image(image_url)
        if not image_data:
            logger.error("Failed to fetch image data")
            return None

        return image_data
    except Exception as e:
        logger.error(f"Error in generate_image: {str(e)}")
        return None

# Enhanced logging setup remains the same...

def validate_base_prompt(prompt):
    """
    Validates the base prompt and provides suggestions for improvement
    """
    if not prompt or prompt.strip() == "":
        return False, "Base prompt cannot be empty. Please describe what you want to generate."

    # Check minimum word count (recommended at least 3 words)
    words = prompt.split()
    if len(words) < 3:
        return False, "Base prompt is too short. Try adding more details (at least 3 words recommended)."

    # Check for common issues
    common_issues = []

    # Check for specific object/subject
    if not any(word.lower() in prompt.lower() for word in ['spaceship', 'car', 'person', 'building', 'landscape', 'animal']):
        common_issues.append("Consider specifying the main subject (e.g., spaceship, person, landscape)")

    # Check for descriptive adjectives
    descriptive_words = ['red', 'large', 'modern', 'ancient', 'futuristic', 'beautiful', 'dark', 'bright']
    if not any(word.lower() in prompt.lower() for word in descriptive_words):
        common_issues.append("Add descriptive adjectives (e.g., modern, futuristic, ancient)")

    # If there are issues but prompt is not invalid
    if common_issues and len(words) >= 3:
        return True, "Suggestions for better results:\n- " + "\n- ".join(common_issues)

    return True, "Prompt looks good!"


def generate_and_display(base_prompt, resolution, aspect_ratio, color_scheme, style,
                         mood, lighting, camera_angle, background, text_overlay, text_position):
    """
    Enhanced wrapper function with text overlay support
    """
    try:
        logger.info("Starting generate_and_display function")

        # Validate base prompt
        is_valid, message = validate_base_prompt(base_prompt)
        if not is_valid:
            return None, message

        # Generate image with text overlay
        image_data = generate_image(
            base_prompt, resolution, aspect_ratio, color_scheme,
            style, mood, lighting, camera_angle, background,
            text_overlay, text_position
        )

        if not image_data:
            logger.error("No image data received from generation")
            return None, "Failed to generate image. Please check logs for more details."

        # Convert to PIL Image
        img = Image.open(BytesIO(image_data))
        logger.info("Successfully converted image data to PIL Image")

        return img, "Image generated successfully!"
    except Exception as e:
        error_msg = str(e)
        logger.error(f"Error converting image data: {error_msg}")
        return None, f"Error processing image: {error_msg}"


# Gradio interface setup
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🎨 Parameterized AI Image Generator with Base Prompt")

    with gr.Row():
        base_prompt_input = gr.Textbox(
            label="Base Prompt",
            placeholder="Describe your base image concept",
            lines=2
        )

    # Define prompt_feedback before using it
    prompt_feedback = gr.Textbox(
        label="Prompt Feedback",
        interactive=False
    )

    # Add text overlay controls
    with gr.Row():
        text_overlay_input = gr.Textbox(
            label="Text Overlay",
            placeholder="Enter text to overlay on image"
        )
        text_position_input = gr.Dropdown(
            label="Text Position",
            choices=["top", "bottom", "left", "right", "center"],
            value="center"
        )

    with gr.Row():
        style_input = gr.Dropdown(
            label="Art Style",
            choices=["Realistic", "Digital Art", "3D Render", "Pencil Sketch"],
            value="Realistic"
        )
        resolution_input = gr.Dropdown(
            label="Resolution",
            choices=["None", "512x512", "1024x1024", "2048x2048"],
            value="1024x1024"
        )
        aspect_ratio_input = gr.Dropdown(
            label="Aspect Ratio",
            choices=["1:1", "16:9", "9:16", "3:2"],
            value="1:1"
        )

    with gr.Row():
        color_input = gr.Textbox(
            label="Color Scheme",
            placeholder="e.g., metallic, red, blue"
        )
        mood_input = gr.Textbox(
            label="Mood",
            placeholder="e.g., dramatic, peaceful, modern"
        )
        lighting_input = gr.Textbox(
            label="Lighting",
            placeholder="e.g., natural, studio, dramatic"
        )

    with gr.Row():
        camera_angle_input = gr.Textbox(
            label="Camera Angle",
            placeholder="e.g., front view, side view, aerial view"
        )
        background_input = gr.Textbox(
            label="Background",
            placeholder="e.g., forest, galaxy, abstract"
        )

    def update_prompt_feedback(prompt):
        is_valid, message = validate_base_prompt(prompt)
        return message

    base_prompt_input.change(
        fn=update_prompt_feedback,
        inputs=[base_prompt_input],
        outputs=[prompt_feedback]
    )

    generate_button = gr.Button("Generate Image")
    image_output = gr.Image(label="Generated Image")
    status_output = gr.Textbox(label="Status", interactive=False)

    generate_button.click(
    fn=generate_and_display,
    inputs=[
        base_prompt_input,
        resolution_input,
        aspect_ratio_input,
        color_input,
        style_input,
        mood_input,
        lighting_input,
        camera_angle_input,
        background_input,
        text_overlay_input,
        text_position_input
    ],
    outputs=[image_output, status_output]
)


if __name__ == "__main__":
    demo.launch(show_error=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://28734e5e720ea47280.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
